In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tkinter as tk
import tkinter.filedialog as fd
import numpy as np

In [3]:
def number_type_from_imagename(image_name):
    i = 0
    for s in image_name:
        if(s.isspace())==False:
            i=i+1
        else:
            break

    name_and_type = image_name[0:i]

    y = 0
    for s in name_and_type:
        if(s.isdigit()):
            y = y+1
    

    image_number = image_name[0:y]
    image_type = image_name[y:i]


    return image_number,image_type

In [6]:
root = tk.Tk()
folder_path = fd.askdirectory(parent=root, title='Folder with images')

cells_path = folder_path + "/Cells_detected.csv"
apoptosis_path = folder_path + "/Apoptotic_Cleared.csv"
necrosis_path = folder_path + "/Necrotic_Cleared.csv"
an_path = folder_path + "/Necrotic_Apoptotic_Cells.csv"

cells_df = pd.read_csv(cells_path)
apoptosis_df = pd.read_csv(apoptosis_path)
necrosis_df = pd.read_csv(necrosis_path)
an_df = pd.read_csv(an_path)

print(np.shape(cells_df))

(46392, 18)
